In [39]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
import datetime
import matplotlib.pyplot as plt
import collections as cc
import seaborn as sns
from collections import Counter
import re
from bs4 import BeautifulSoup
from konlpy.tag import Okt


data1_dir = "../data1.csv"
data2_dir = "../data2.csv"
data3_dir = "../data3.csv"
data4_dir = "../data4.csv"
### 전처리 완료 후 에어포스에 대해서 처리한 데이터 -> processed_data
data_dir='../processed_data.csv'
score_dir='../score_data.csv'

In [40]:
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    return stopwords

def preprocessing(description, remove_stopwords =False):
    description_text=BeautifulSoup(description,'html5lib').get_text()
    description_text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', description_text)
    description_text = re.sub(r'\s+', ' ', description_text).strip()
    okt = Okt()
    description_text = description_text.replace('사 용감', '사용감')
    description_text = description_text.replace('사용 감', '사용감')
    word_description=okt.morphs(description_text, stem=False) 

    if remove_stopwords:
        korean_stop_words = load_stopwords('stopwords-ko.txt')
        word_description=[token for token in word_description if not token in korean_stop_words]
    return word_description

In [41]:
data=pd.read_csv(data1_dir,encoding='cp949')
data2=pd.read_csv(data2_dir,encoding='cp949')
data3=pd.read_csv(data3_dir,encoding='cp949')
data4=pd.read_csv(data4_dir,encoding='cp949')

C:\Users\sunwoong\AppData\Local\Temp\ipykernel_5332\1075909164.py:1: DtypeWarning: Columns (22,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(data1_dir,encoding='cp949')


In [44]:
name_kor_counts = data['name_kor'].value_counts()


In [45]:
name_kor_counts[:15]

name_kor
나이키 에어포스 1 '07 로우 화이트                          1176
뉴발란스 992 메이드 인 USA 그레이                          995
뉴발란스 993 메이드 인 USA 그레이                          993
나이키 에어포스 1 '07 WB 플랙스                           990
나이키 덩크 로우 레트로 블랙                                982
컨버스 척 70 하이 블랙                                  822
팀버랜드 프리미엄 6인치 워터프루프 부츠 위트 누벅                    694
조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루     615
아디다스 삼바 OG 클라우드 화이트                             609
아디다스 슈퍼스타 코어 블랙 화이트                             597
컨버스 척 70 로우 블랙                                  590
나이키 에어포스 1 '07 로우 트리플 블랙                        550
아식스 젤 카야노 14 크림 블랙                              547
아디다스 삼바 OG 블랙 화이트 검                             512
나이키 SB 덩크 로우 프로 시카고                             490
Name: count, dtype: int64

In [46]:
filtered_data = data[data['name_kor'].str.contains("나이키 에어포스 1 '07 로우 화이트", na=False)]


In [47]:
filtered_data = filtered_data[~filtered_data['id'].isin(data3['pid'])]

In [48]:
filtered_data.head()

,id,status,create_date,name,keyword,price,flag_used,flag_taekpo,flag_exchg,qty,...,uid,is_active,created_at,setl_done_date,purchase_confirm_date,total_price,talk_user_count,first_talk_date,sell_count,register_prod_count
860,8843435ba249eda0d211b860b1eb213a,3,2023-12-19 20:13,240 나이키 에어포스1 로우 올백 화이트,240,20000,20,0,0,0,...,cc2d6707a650836c13cebe3549d63ba4,1.0,2022-09-30 13:58,2023-12-20 15:54,2023-12-22 23:24,23000.0,1.0,2023-12-20,871,1
1028,0bc912bd886998ebc4c0c5ba6b362700,3,2024-04-05 13:57,나이키 에어포스,"나이키,에어포스,275",75000,20,1,0,0,...,NaN,NaN,NaN,2024-04-05 16:53,2024-04-18 14:07,87620.0,2.0,2024-04-05,1,1
1128,ac2a3dc7a07a0927e85ff6ea0820051c,3,2024-01-17 23:08,에어포스1,나이키,140000,20,1,0,0,...,NaN,NaN,NaN,2024-01-18 18:20,2024-01-18 18:46,144900.0,1.0,2024-01-18,14,1
1272,ac78a060c12396c566e9eb7de105243c,3,2024-01-22 16:52,나이키 에어포스1 올백 275 팝니다,NaN,60000,21,1,0,0,...,NaN,NaN,NaN,NaN,2024-01-31 17:10,60000.0,1.0,2024-01-22,3,1
1969,61b88b1a528f28b4032310703a2b1dc7,3,2024-02-14 15:34,나이키 에어포스 1 올백 240,"나이키,에어포스,에어포스1,나이키포스",69000,21,0,0,0,...,NaN,NaN,NaN,NaN,2024-03-05 19:51,69000.0,5.0,2024-02-15,2,1


In [49]:
filtered_data2 = filtered_data.dropna(subset=['setl_done_date', 'purchase_confirm_date'], how='all')


In [50]:
filtered_data2.to_csv('Airfocre_tot.csv',encoding='cp949',index=False)

## 에어포스 이미지 다운로드

In [5]:
import os
import requests
from PIL import Image
from io import BytesIO
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# CSV 파일 경로
file_path = 'hash_pid_with_imageUrls.csv'
file_path2 = 'Airfocre_tot.csv'
#file_path = 'sample2.csv'
# CSV 파일 읽기
data1 = pd.read_csv(file_path)
data2 = pd.read_csv(file_path2,encoding='cp949')
filtered_pid = data1[data1['hash_pid'].isin(data2['id'])]

# 이미지 다운로드 및 저장 함수
def download_and_save_image(image_url, pid, image_number):
    try:
        response = requests.get(image_url, timeout=10)
        if response.status_code == 200:
            print("hi")
            image = Image.open(BytesIO(response.content))
            image = image.convert('RGB')
            directory = f'AIRFORCE_classification25'
            if not os.path.exists(directory):
                os.makedirs(directory)
                print("hi")
            file_name = os.path.join(directory, f'{pid}_{image_number}.jpg')
            image.save(file_name, 'JPEG')
    except Exception as e:
        print(f"Error downloading {image_url}: {e}")

# 멀티스레딩으로 이미지 다운로드
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
    for index, row in filtered_pid.iterrows():
        pid = row['hash_pid']
        for i in range(1, 13):
            image_url_column = f'imageUrl{i}'
            if pd.notna(row[image_url_column]):
                futures.append(executor.submit(download_and_save_image, row[image_url_column], pid, i))
    
    # tqdm을 사용하여 진행 상황 표시
    for _ in tqdm(ThreadPoolExecutor(max_workers=10).map(lambda x: x.result(), futures), total=len(futures), desc="Downloading images"):
        pass

In [6]:
import os
import shutil

# 데이터 폴더의 경로 설정
data_folder_path = './Airforce_white'

# data 폴더 내 모든 pid 폴더들 가져오기
pid_folders = [os.path.join(data_folder_path, pid_folder) for pid_folder in os.listdir(data_folder_path) if os.path.isdir(os.path.join(data_folder_path, pid_folder))]

# 각 pid 폴더를 순회하며 이미지 파일을 data 폴더로 이동
for pid_folder in pid_folders:
    for filename in os.listdir(pid_folder):
        file_path = os.path.join(pid_folder, filename)
        if os.path.isfile(file_path):
            # 이동할 경로 설정, 중복 파일명 처리
            new_file_name = f"{os.path.basename(pid_folder)}_{filename}" if os.path.exists(os.path.join(data_folder_path, filename)) else filename
            new_file_path = os.path.join(data_folder_path, new_file_name)
            shutil.move(file_path, new_file_path)

    # 모든 파일을 이동한 후, 빈 폴더 삭제
    os.rmdir(pid_folder)

print("모든 이미지를 data 폴더로 이동 완료!")


모든 이미지를 data 폴더로 이동 완료!
